In [187]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re

In [188]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [189]:
import json 
# with open ('2019_public_comment.json','w') as f1:
#     json.dump(dic,f1)
# with open ('2020_may30_lexiconV1.json') as f1:
#     dic = json.load(f1)
# V4_1 80%
# V4_2 70%  # good!!! 
# V4_3 60% # good too!!!
# V4_4 50%
# V4_5  combine 60% and 70%
with open ('2020_june_lexiconV4_5.json') as f1:
    dic = json.load(f1)

In [190]:
file1 = '2020_FB_Pres_500_Ads_Topic_GOLD_v2.xlsx'
file2 = '2020_FB_Pres_1000_Posts_Topic_GOLD_v2.xlsx'
file3 = '2020_FB_Pres_Posts_500_Topic_GOLD_v1.xlsx'
dirs = 'C:\\Users\\wangtao\\Documents\\Illumination\\data_set(2020june)\\'


# df1 = pd.read_excel(dirs+file3)
# df1 = pd.read_excel('Facebook Ads Analysis.xlsx') # this data is coming from the facebook ad 2016 (the )
df1 = pd.read_csv('FB_2000.csv')

In [191]:
df1.columns

Index(['text', 'Gold Label'], dtype='object')

In [155]:
# df1 = df1[['ad_creative_body','GOLD']]
# df1 = df1[['text','GOLD']]

In [156]:
df1.rename(columns={'GOLD':'Gold Label'},inplace=True)
# df1.rename(columns={'ad_creative_body':'text'}, inplace=True)

In [192]:
df1['Gold Label'].isnull().sum()
#three null?
#974 in totall

0

In [193]:
def filter_text(x):
    url = 'http[s]?://\S+'
    x = re.sub(url,'',x)
    x = re.sub("[^\w\s]",' ',x)
    x = re.sub("\s+",' ',x)
    
    ls=[w.lower() for w in x.split()] # if w.lower() not in stopword]
    
    return ' '.join(ls)

In [194]:
dic1 = dic

In [195]:
df1['text'] = df1['text'].astype(str).apply(lambda x: filter_text(x))

In [196]:
df1.loc[0:10,'text']

0     last quarter bernie sanders pete buttigieg and...
1     every one of the democratic candidates remaini...
2     a young woman came up to me after my speech in...
3     como presidenta elizabeth proporcionarн_a el m...
4     una cosa queda clara ahorrar para una jubilaci...
5     думдуцi think it is really important that as w...
6     as i travel the country talking to workers at ...
7     думthe senator praised unions as a force for a...
8     дум her plan also highlights the need for a ne...
9     думдуцthe burdens of student debt are not dist...
10    думдуцthe real danger we face as democrats is ...
Name: text, dtype: object

In [197]:
df1['Gold Label']=df1['Gold Label'].apply(lambda x:','.join([m.strip() for m in sorted(x.lower().split(','))]))

In [198]:
def find_words(x,lexicon):
    topics= lexicon.keys()
    
    doc = nlp(x)
    words= []
    for t in topics:
        terms= lexicon[t]
        patterns = [nlp.make_doc(text) for text in terms]
        matcher.add("TerminologyList", None, *patterns)
        matches = matcher(doc)
        for match_id, start,end in matches:
            span = doc[start:end]
            words.append(span.text)
    if words:
        words = list(set(words))
        return ','.join(words)
    else:
        return('no words')
        

In [199]:
# find the words in each message
def find_topic(x,lexicon):
    topics= lexicon.keys()
    
    if x=='no words':
        return ''
    
    if x != 'no words': 
        words = x.split(',')
        labels = []
        
        for t in topics:            
            terms = lexicon[t]
            if set(words)&set(terms):
                #print(t)
                #print(set(words)&set(terms))
                labels.append(t)
                
                #l = sorted(labels)
        
        return  ','.join(sorted(labels))
                
        #return ','.join(labels)
            

In [200]:
# get the new labels from classifer2 which depending on the updated lexicon
df1['words'] = df1['text'].astype(str).apply(lambda x: find_words(x,dic1))
#df1.rename(columns={'m_label':'words'},inplace=True)

In [201]:
df1.tail(10)

,text,Gold Label,words
1989,let me be clear if we do not protect the worki...,"covid,education,economic","student,economy,student loan"
1990,thank you to greenpeace for recognizing the st...,environmental,climate
1991,with your help we ll defeat donald trump and w...,no topic,no words
1992,i m the only candidate who will say this i wil...,environmental,"climate change,climate"
1993,new york post editorial board adam schiff is w...,governance,impeachment
1994,thank you rashad robinson for having me on the...,"economic,social cultural,safety","black,criminal,racial,wealth,criminal justice,..."
1995,louis vuitton has invested in the american wor...,economic,"workers,american workers"
1996,upper income people have done disproportionate...,economic,"tax plan,economy,tax"
1997,as usual with this general motors things just ...,"health,covid","ventilators,invoke"
1998,black and brown families value education the s...,"economic,social cultural,education","segregated,black,education"


In [202]:
df1['m_label'] = df1['words'].apply(lambda x: find_topic(x,dic))

In [203]:
df1['m_label'] = df1['m_label'].apply(lambda x: 'no topic' if x=='' else x)

In [204]:
df1.tail(5)

,text,Gold Label,words,m_label
1994,thank you rashad robinson for having me on the...,"economic,social cultural,safety","black,criminal,racial,wealth,criminal justice,...","economic,safety,social cultural"
1995,louis vuitton has invested in the american wor...,economic,"workers,american workers",economic
1996,upper income people have done disproportionate...,economic,"tax plan,economy,tax",economic
1997,as usual with this general motors things just ...,"health,covid","ventilators,invoke",covid
1998,black and brown families value education the s...,"economic,social cultural,education","segregated,black,education","education,social cultural"


In [135]:
# df1.to_csv('2020july13_FB_2000_LabelResult.csv',index=False)

### Make the matrix and calculate precision, recall F1

In [136]:
# df1 = pd.read_csv('2020june27_FB_2000_LabelResult.csv')

In [205]:
df1.shape

(1999, 4)

In [206]:
# df2 = df1[['text','Gold Label', 'm_label']]
df2 = df1[['Gold Label', 'm_label']]

In [207]:
l_topic = dic1.keys()

pred_label = [x+'_pred' for x in l_topic]

pred_label
    

['covid_pred',
 'economic_pred',
 'education_pred',
 'environmental_pred',
 'foreign policy_pred',
 'governance_pred',
 'health_pred',
 'immigration_pred',
 'military_pred',
 'safety_pred',
 'social cultural_pred',
 'social programs_pred']

In [208]:
n=len(df2)
# n=1000
data0 = {x:np.zeros(n,dtype=int) for x in l_topic}
data1 = {x:np.zeros(n,dtype=int) for x in pred_label}

table0= pd.DataFrame(data0)

tabel1 = pd.DataFrame(data1)

c_df = pd.concat([df2,table0,tabel1],axis=1)

c_df.tail(10)


data0 = {x:np.zeros(n,dtype=int) for x in l_topic}
data1 = {x:np.zeros(n,dtype=int) for x in pred_label}

table0= pd.DataFrame(data0)

tabel1 = pd.DataFrame(data1)

c_df = pd.concat([df2,table0,tabel1],axis=1)

c_df.tail(10)

,Gold Label,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,...,education_pred,environmental_pred,foreign policy_pred,governance_pred,health_pred,immigration_pred,military_pred,safety_pred,social cultural_pred,social programs_pred
1989,"covid,education,economic","economic,education,social programs",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1990,environmental,environmental,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1991,no topic,no topic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1992,environmental,environmental,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1993,governance,governance,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1994,"economic,social cultural,safety","economic,safety,social cultural",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1995,economic,economic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,economic,economic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,"health,covid",covid,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,"economic,social cultural,education","education,social cultural",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
 # in Ben and Juliar's annotation, they made some spelling mistakes of the labels like,
# typing the "social programs" as "social program", type "environment" as "environmental" etc.
# so I made this excel and correct the spelling manually.
# next step is labeling the matrix, so you may got key error because of the spelling mistakes

# c_df.to_excel('calculate_FB2000_f1.xlsx',index=False)

In [175]:
# c_df = pd.read_excel('calculate_f1.xlsx')

In [209]:
c_df = c_df[c_df['Gold Label']!='']
c_df.reset_index(drop=True,inplace=True)

In [210]:
for index, row in c_df.iterrows():
    #print(index)
    #print(row)
    try:
        if len(row['Gold Label'].split(',')) ==1 and row['Gold Label']!= 'no topic':
            c_df[row['Gold Label'].strip()][index]=1

        if len(row['Gold Label'].split(','))>1:
            for w in row['Gold Label'].split(','):
                #print(index)
                c_df[w.strip()][index]=1

        if len(row['m_label'].split(',')) ==1 and row['m_label']!= 'no topic':
            c_df[row['m_label']+'_pred'][index]=1

        if len(row['m_label'].split(','))>1:
            for w in row['m_label'].split(','):
                c_df[w+'_pred'][index]=1
        else: 
            pass
    
    except KeyError: 
        print (index)
        print(row['Gold Label'])
        pass
        

C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\wangtao\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\wangtao\Anaconda3\lib\site-p

In [211]:
c_df.head()

,Gold Label,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,...,education_pred,environmental_pred,foreign policy_pred,governance_pred,health_pred,immigration_pred,military_pred,safety_pred,social cultural_pred,social programs_pred
0,no topic,no topic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,no topic,no topic,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"education,social programs","education,social programs",0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,social programs,social programs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,social programs,social programs,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


## make the one_hot_post file for new words extration

In [25]:
c_df1 = c_df.iloc[:,0:15]

In [27]:
c_df1.drop(columns=['Gold Label','m_label'],inplace=True)

In [29]:
c_df1.to_excel('FB2000_one_hot_post.xlsx',index=False)

## caculate F1

In [212]:
C_df = c_df.drop(columns=['Gold Label', 'm_label'])

In [213]:
C_df.shape

(1999, 24)

In [214]:
x = C_df.columns.get_loc('governance') # get the column index
x

5

In [221]:
def confusion_matrix1(vectorg, vectpre):
    tp = 0 
    tn = 0 
    fp = 0
    fn = 0
    for i in range(len(vectorg)):
        # print(vectorg[i])
        if vectorg[i] == 1 and vectpre[i] == 1:
            tp = tp + 1
        elif vectorg[i] == 1 and vectpre[i] == 0:
            fn = fn + 1
        elif vectorg[i] == 0 and vectpre[i] == 0:
            tn = tn + 1
        elif vectorg[i] == 0 and vectpre[i] == 1:
            fp = fp + 1
    print('tp =', tp ,'fp =', fp ,'tn =',tn,'fn =', fn)
    print("the number of true label："+ str(tp+fn))
    n=tp+fn
    #
    return (tp, fp, tn, fn,n)
        
def precision_recall(vectorg, vectpre):
    #print(vectorg[0],vectpre[0])
    try:
        tp, fp, tn, fn,n = confusion_matrix1(vectorg, vectpre)
        precision = tp/(tp + fp)
        print ('precision = ' + str(precision))
        recall = tp/ (tp + fn)
        print ('recall = ' + str(recall))
        if precision!=0 and recall!=0:
            f1 = 2/((1/precision)+(1/recall))
            print ('f1 =' + str(f1))
        if precision==0 or recall==0:
            print ('ZeroDivisionError')
            f1 = 'ZeroDivisionError'
    
        return [precision,recall,f1,n]
    
    except ZeroDivisionError:
        precision=0
        recall=0
        f1=0
        return [precision,recall,f1,n]
        print ('ZeroDivisionError')
    

In [222]:
categories = list(C_df.columns.values)
categories = categories[0:12]        
print (categories)


['covid', 'economic', 'education', 'environmental', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social cultural', 'social programs']


In [223]:
result = {}
for category in categories:
    x = C_df.columns.get_loc(category)
    print(category.strip())
    r_dic = {category.strip():precision_recall(C_df.iloc[:,x],C_df.iloc[:,x+12])}
    result.update(r_dic)
    print('\n')

covid
tp = 28 fp = 2 tn = 1957 fn = 12
the number of true label：40
precision = 0.9333333333333333
recall = 0.7
f1 =0.8


economic
tp = 257 fp = 32 tn = 1581 fn = 129
the number of true label：386
precision = 0.889273356401384
recall = 0.6658031088082902
f1 =0.7614814814814815


education
tp = 106 fp = 20 tn = 1857 fn = 16
the number of true label：122
precision = 0.8412698412698413
recall = 0.8688524590163934
f1 =0.8548387096774194


environmental
tp = 101 fp = 8 tn = 1860 fn = 30
the number of true label：131
precision = 0.926605504587156
recall = 0.7709923664122137
f1 =0.8416666666666666


foreign policy
tp = 40 fp = 4 tn = 1925 fn = 30
the number of true label：70
precision = 0.9090909090909091
recall = 0.5714285714285714
f1 =0.7017543859649122


governance
tp = 102 fp = 10 tn = 1798 fn = 89
the number of true label：191
precision = 0.9107142857142857
recall = 0.5340314136125655
f1 =0.6732673267326732


health
tp = 139 fp = 13 tn = 1772 fn = 75
the number of true label：214
precision = 0.

In [224]:
re_df = pd.DataFrame(result)
re_df = re_df.T
re_df.rename(columns={0:'Precision',1:'Recall',2:'F1_Score',3:'number of messages'},inplace=True)
re_df

,Precision,Recall,F1_Score,number of messages
covid,0.933333,0.700000,0.800000,40.0
economic,0.889273,0.665803,0.761481,386.0
education,0.841270,0.868852,0.854839,122.0
environmental,0.926606,0.770992,0.841667,131.0
foreign policy,0.909091,0.571429,0.701754,70.0
governance,0.910714,0.534031,0.673267,191.0
health,0.914474,0.649533,0.759563,214.0
immigration,0.913580,0.822222,0.865497,90.0
military,0.900000,0.360000,0.514286,50.0
safety,0.877863,0.701220,0.779661,164.0


In [225]:
# re_df.to_csv('2020_FB_500postv1_CNN_V34.csv')
re_df.to_csv('2020_FB_2000_CNN_V4_5.csv')